In [2]:
import queue

import numpy as np
from torchvision.datasets import MNIST

def download_mnist(is_train: bool):
    dataset = MNIST(root='./data',
                    transform=lambda x: np.array(x).flatten(),
                    download=True,
                    train=is_train)
    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image)
        mnist_labels.append(label)
        
    return mnist_data, mnist_labels

In [4]:
train_X, train_Y = download_mnist(is_train=True)
test_X, test_Y = download_mnist(is_train=False)

# Normalizarea inputului


In [5]:
def normalize_input(lst: list[int]):
    return [x / 255 for x in lst]

train_X = normalize_input(train_X)
test_X = normalize_input(test_X)

In [6]:
def label_encoding(labels: list[int]) -> list[list[int]]:
    return [[1 if label == i else 0 for i in range(10)] for label in labels]

train_Y = label_encoding(train_Y)
test_Y = label_encoding(test_Y)

### Salvare/Incarcare date
Pentru a nu descărca de atâtea ori, salvam si incarcam de pe disc datele deja prelucrate

In [7]:
np.savez('mnist_data.npz',
         train_X=train_X,
         train_Y=train_Y,
         test_X=test_X,
         test_Y=test_Y)

In [1]:
import numpy as np

data = np.load('mnist_data.npz')
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

### Funcția de antrenare a unui batch

In [3]:
def classify(y: np.ndarray[10], label: np.ndarray[10]):
    return np.argmax(y) == np.argmax(label)

def softmax(y: np.ndarray[10]):
    return np.exp(y) / np.sum(np.exp(y))

def cross_entropy(y: np.ndarray[10], label: np.ndarray[10]):
    return - np.sum(label * np.log(y))

In [4]:
def train_thread(train_set, weights: np.ndarray, biases: np.ndarray, learning_rate: float, result_queue: queue.Queue):
    thread_weights = np.zeros(weights.shape)
    thread_biases = np.zeros(biases.shape)
    batches = [train_set[i:i + 100] for i in range(0, len(train_set), 100)]
    for batch in batches:
        temp_weights, temp_biases = train(batch, weights, biases, learning_rate)
        thread_weights = thread_weights + temp_weights
        thread_biases = thread_biases + temp_biases
    result_queue.put((thread_weights, thread_biases))

In [5]:
def train(train_set, weights: np.ndarray, biases: np.ndarray, learning_rate):
    temp_weights = np.zeros(weights.shape)
    temp_biases = np.zeros(biases.shape)
    for x, label in train_set:
        z = x.dot(weights) + biases
        y = softmax(z)
        if not classify(y, label):
            temp_weights = temp_weights + learning_rate * (label - y) * np.transpose(x)
            temp_biases = temp_biases + learning_rate * (label - y)
    return temp_weights, temp_biases

In [ ]:
import threading

weights = np.zeros((784, 10))
biases = np.zeros(10)
alpha = 0.01
epochs = 3
num_threads = 8

train_data = list(zip(train_X, train_Y))
train_data_len = len(train_data)
thread_set_size = train_data_len // num_threads

for epoch in range(epochs):
    threads = []
    result_queue = queue.Queue()
    thread_batches = [train_data[i:i + thread_set_size] for i in range(0, train_data_len, thread_set_size)]
    for i in range(num_threads):
        thread = threading.Thread(target=train_thread, args=(train_data, weights, biases, alpha, result_queue))
        threads.append(thread)
        thread.start()
        
    for t in threads:
        t.join()
        
    for i in range(num_threads):
        temp_weights, temp_biases = result_queue.get()
        weights = weights + temp_weights
        biases = biases + temp_biases
    
    print(f'Epoch {epoch + 1} done')
    
# Save the model
np.savez('model.npz', weights=weights, biases=biases)